## *PyTorch para Deep Learning*

O código abaixo é baseado no curso/livro on-line "Zero to Mastery Learn PyTorch for Deep Learning", disponível em <https://www.learnpytorch.io/>

## 01. PyTorch Workflow Fundamentals

In [1]:
# dicionário para guardar o processo de trabalho para a criação de um modelo de ML, 
# usado como uma referência para o que será feito ao longo desse processo

what_we_are_covering = { 1: "data (prepare and load)",
             2: "build model",
             3: "fitting the model to data (training)",
             4: "making predictions and evaluating a model (inference)",
             5: "saving and loading a model",
             6: "putting it all together"          
}

In [2]:
# importação de torch, torch.nn e matplotlib
# segundo o livro: "nn stands for neural network and this package contains the building blocks for creating neural networks in PyTorch"

import torch
from torch import nn
import matplotlib.pyplot as plt
import torch.version

# Para checar a versão do Pytorch
torch.__version__

'2.5.1+cu124'

In [4]:
# segundo o livro:

# "Machine learning is a game of two parts:
# Turn your data, whatever it is, into numbers (a representation).
# Pick or build a model to learn the representation as best as possible."

# diante disso, cria-se dados simples para a partir deles montar um modelo de *regressão linear*
# ou seja, eu tenho os dados e vou fazer com que os PARÂMETROS (não os dados/entrada, pois isso é impossível)
# do modelo sejam alterados para melhor representar o padrão presente ali

# crio parâmetros conhecidos - weight * x + bias (ou a * x + b)
weight = 0.7
bias = 0.3

# crio os dados para a simulação de uma conjunto de dados
start = 0
end = 1
step = 0.02 # ou seja, o meu conjunto de dados tem 50 números
X = torch.arange(start, end, step).unsqueeze(dim=1) # X representa a entrada (50 números)/características dos dados/features
y = weight * X + bias # y representa a saída original que eu vou tentar prever, é o rótulo/label que mostra qual é o resultado dos dados de entrada

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))